In [27]:
import os
import h5py
import numpy as np
import pandas as pd
import glob
import itertools
import statistics


def filterOutliers(row): 
    elevs = [row.elev1, row.elev2, row.elev3, row.elev4, row.elev5, row.elev6]
    sd = statistics.pstdev(elevs)
    elevRange = elevs.max() - elevs.min()
    return all((x < (sd + 2) and x > (sd - 2)) for x in elevs) and elevRange < 2
        

gedi_dfs = []

f = open("h5S3files.txt", "r")

for h5file in f.readlines():
    cpS3url = "aws s3 cp s3://servir-public/gedi/peru/" + h5file " ./"
    os.system(cpS3url)
    ###READ DATA
    gediL2A = h5py.File(h5file, 'r')  
    #beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]
    gediL2A_objs = []
    gediL2A.visit(gediL2A_objs.append)                                           # Retrieve list of datasets
    gediSDS = [o for o in gediL2A_objs if isinstance(gediL2A[o], h5py.Dataset)]  # Search for relevant SDS inside data file

    ###FITLER DATA 
    rh_cols = []
    for i in range(101):
        rh_cols.append('rh'+str(i))

    beam_dfs = []
        
    for highBeam in beamNames:
        gedi_beam_data = {}
        try:
            gedi_beam_data["lats"] = gediL2A[f'{highBeam}/lat_lowestmode'][()]
            gedi_beam_data["long"] = gediL2A[f'{highBeam}/lon_lowestmode'][()]
            gedi_beam_data["shotNumber"] = gediL2A[f'{highBeam}/shot_number'][()]
            gedi_beam_data["quality"] = gediL2A[f'{highBeam}/quality_flag'][()]
            gedi_beam_data["solar_elev"] = gediL2A[f'{highBeam}/solar_elevation'][()]
            gedi_beam_data["sensitivity"] = gediL2A[f'{highBeam}/sensitivity'][()]
            #gedi_beam_data["rh"] = gediL2A[f'{highBeam}/rh'][()]
            gedi_beam_data["elevLow"] = gediL2A[f'{highBeam}/elev_lowestmode'][()]

            gedi_beam_data["delta_time"] = gediL2A[f'{highBeam}/delta_time'][()]
            gedi_beam_data["energy_total"] = gediL2A[f'{highBeam}/energy_total'][()]
                
            gedi_beam_data["rx_gbias"] = gediL2A[f'{highBeam}/rx_1gaussfit/rx_gbias'][()]
            gedi_beam_data["rx_gamplitude_error"] = gediL2A[f'{highBeam}/rx_1gaussfit/rx_gamplitude_error'][()]
                
            gedi_beam_data["rx1_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a1/energy_sm'][()]
            gedi_beam_data["rx2_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a2/energy_sm'][()]
            gedi_beam_data["rx3_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a3/energy_sm'][()]
            gedi_beam_data["rx4_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a4/energy_sm'][()]
            gedi_beam_data["rx5_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a5/energy_sm'][()]
            gedi_beam_data["rx6_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a6/energy_sm'][()]
             
            gedi_beam_data["rx1_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a1/lastmodeenergy'][()]
            gedi_beam_data["rx2_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a2/lastmodeenergy'][()]
            gedi_beam_data["rx3_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a3/lastmodeenergy'][()]
            gedi_beam_data["rx4_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a4/lastmodeenergy'][()]
            gedi_beam_data["rx5_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a5/lastmodeenergy'][()]
            gedi_beam_data["rx6_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a6/lastmodeenergy'][()]
                
            gedi_beam_data["elev1_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a1'][()]
            gedi_beam_data["elev2_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a2'][()]
            gedi_beam_data["elev3_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a3'][()]
            gedi_beam_data["elev4_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a4'][()]
            gedi_beam_data["elev5_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a5'][()]
            gedi_beam_data["elev6_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a6'][()]
                
            gedi_beam_data["elev1_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a1'][()]
            gedi_beam_data["elev2_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a2'][()]
            gedi_beam_data["elev3_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a3'][()]
            gedi_beam_data["elev4_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a4'][()]
            gedi_beam_data["elev5_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a5'][()]
            gedi_beam_data["elev6_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a6'][()]
             
            gedi_beam_data["elev1"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a1'][()]
            gedi_beam_data["elev2"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a2'][()]
            gedi_beam_data["elev3"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a3'][()]
            gedi_beam_data["elev4"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a4'][()]
            gedi_beam_data["elev5"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a5'][()]
            gedi_beam_data["elev6"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a6'][()]
            rh = gediL2A[f'{highBeam}/rh'][()]
            gedi_beam_data["beam"] = np.full(len(gedi_beam_data['shotNumber']), highBeam)
                #raw_df = pd.DataFrame({shots, 'Beam':np.full(len(shots), highBeam), 'Longitude': lons, 'Latitude': lats,
                 #                        'Quality': quality, 'Solar_Elevation': solar_elev, 'Sensitivity': sensitivity, "elev_lowestmode": elevLow,
                 #                        'ElevationA1': elev1, 'ElevationA2': elev2, 'ElevationA3': elev3, 'ElevationA4': elev4, 'ElevationA5': elev5, 'ElevationA6': elev6})
                
            raw_df = pd.DataFrame(gedi_beam_data)
            raw_df[rh_cols] = pd.DataFrame(rh, index= raw_df.index)
            display(raw_df)
            raw_df = raw_df.dropna()
            beam_dfs.append(raw_df)
        except Exception as e:
            print(e)
            break

        

    raw_df = pd.concat(beam_dfs)
    filtered = raw_df[(raw_df.sensitivity >= 0.9)  & (raw_df.solar_elev < 0) ] #& (raw_df.Quality == 1)
    GEDI_DF = filtered.apply(filterOutliers, axis=1)


    GEDI_DF = GEDI_DF[GEDI_DF.rh95 > 0]
    gedi_dfs.append(GEDI_DF)
    os.remove(h5file)

df = pd.concat(gedi_dfs)

df.to_csv("processedShots.csv", index = False)


HERE


C:\Users\bella\Anaconda3\envs\ndvi\lib\site-packages\pandas\core\frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,lats,long,shotNumber,quality,solar_elev,sensitivity,elevLow,delta_time,energy_total,rx_gbias,...,rh91,rh92,rh93,rh94,rh95,rh96,rh97,rh98,rh99,rh100
0,-8.488011,-75.429546,20190500400256868,1,9.796640,0.988690,311.579132,4.112001e+07,14406.093750,210.758499,...,25.540001,25.840000,26.180000,26.590000,27.040001,27.490000,27.980000,28.500000,29.100000,30.410000
1,-8.488431,-75.429240,20190500400256869,1,9.796215,0.989991,304.010315,4.112001e+07,14651.285156,206.687851,...,34.490002,34.790001,35.169998,35.580002,36.070000,36.630001,37.270000,38.090000,39.209999,40.669998
2,-8.488849,-75.428937,20190500400256870,1,9.795790,0.987630,314.832214,4.112001e+07,14488.989258,205.764771,...,24.040001,24.830000,25.799999,26.700001,27.450001,28.350000,29.510000,32.320000,36.099998,37.939999
3,-8.489265,-75.428635,20190500400256871,1,9.795365,0.986883,338.980103,4.112001e+07,13663.409180,206.497406,...,22.430000,22.770000,23.110001,23.480000,23.930000,24.459999,25.129999,25.920000,26.740000,28.010000
4,-8.489687,-75.428327,20190500400256872,1,9.794936,0.984735,311.917114,4.112001e+07,13876.000977,207.556839,...,29.549999,29.889999,30.219999,30.559999,30.969999,31.420000,31.910000,32.509998,33.330002,34.830002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,-11.324044,-73.344352,20190500400263609,0,6.897656,-0.168351,3118.358887,4.112006e+07,362.579620,204.729218,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3730,-11.324460,-73.344042,20190500400263610,0,6.897229,2.103128,3118.270508,4.112006e+07,-443.096619,204.485687,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3731,-11.324876,-73.343731,20190500400263611,0,6.896801,165.825928,3118.936035,4.112006e+07,-2.767807,205.066238,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3732,-11.325293,-73.343421,20190500400263612,0,6.896373,1.692506,3118.895020,4.112006e+07,-705.831299,204.272110,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


C:\Users\bella\Anaconda3\envs\ndvi\lib\site-packages\pandas\core\frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,lats,long,shotNumber,quality,solar_elev,sensitivity,elevLow,delta_time,energy_total,rx_gbias,...,rh91,rh92,rh93,rh94,rh95,rh96,rh97,rh98,rh99,rh100
0,-8.488536,-75.435718,20190600400787988,1,9.802853,0.980503,303.061523,4.112001e+07,10028.080078,229.842163,...,20.780001,21.120001,21.459999,21.799999,22.170000,22.540001,22.990000,23.440001,24.040001,24.98
1,-8.488957,-75.435412,20190600400787989,1,9.802425,0.977093,286.621155,4.112001e+07,9957.916992,230.686829,...,30.260000,30.639999,31.049999,31.500000,31.950001,32.360001,32.770000,33.220001,33.860001,34.91
2,-8.489370,-75.435112,20190600400787990,0,9.801994,0.963537,329.792694,4.112001e+07,13851.857422,229.552429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,-8.489792,-75.434805,20190600400787991,1,9.801575,0.982984,305.223145,4.112001e+07,13404.996094,229.706802,...,25.879999,26.100000,26.330000,26.549999,26.809999,27.080000,27.379999,27.709999,28.160000,29.32
4,-8.490210,-75.434502,20190600400787992,1,9.801148,0.987572,312.339905,4.112001e+07,14420.786133,230.700150,...,19.440001,19.620001,19.879999,20.150000,20.450001,20.820000,21.230000,21.760000,22.469999,24.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3698,-11.324999,-73.350254,20190600400794734,0,6.903424,-0.063902,3102.375977,4.112006e+07,428.804871,228.905090,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3699,-11.325416,-73.349944,20190600400794735,0,6.902996,-33.689732,3102.341064,4.112006e+07,14.560089,228.872681,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3700,-11.325836,-73.349630,20190600400794736,0,6.902564,0.786530,3080.091553,4.112006e+07,2289.742920,228.497986,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3701,-11.326249,-73.349322,20190600400794737,0,6.902139,1.541115,3102.413574,4.112006e+07,-1023.746521,226.939667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


C:\Users\bella\Anaconda3\envs\ndvi\lib\site-packages\pandas\core\frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,lats,long,shotNumber,quality,solar_elev,sensitivity,elevLow,delta_time,energy_total,rx_gbias,...,rh91,rh92,rh93,rh94,rh95,rh96,rh97,rh98,rh99,rh100
0,-8.488213,-75.445887,20190800400262164,1,9.811746,0.969599,264.538055,4.112001e+07,13398.401367,257.393158,...,28.950001,29.209999,29.510000,29.780001,30.110001,30.450001,30.830000,31.350000,32.060001,33.450001
1,-8.488632,-75.445583,20190800400262165,1,9.811320,0.984727,269.624817,4.112001e+07,12801.159180,256.038635,...,26.260000,26.629999,27.120001,27.750000,28.389999,29.059999,29.850000,31.049999,32.470001,34.009998
2,-8.489051,-75.445278,20190800400262166,1,9.810894,0.988198,271.171448,4.112001e+07,13805.334961,254.934799,...,34.980000,35.509998,36.000000,36.439999,36.860001,37.270000,37.720001,38.320000,39.180000,40.529999
3,-8.489470,-75.444973,20190800400262167,1,9.810468,0.986336,267.321686,4.112001e+07,14308.855469,254.837646,...,43.000000,43.230000,43.490002,43.709999,43.970001,44.200001,44.500000,44.869999,45.360001,46.630001
4,-8.489890,-75.444668,20190800400262168,1,9.810040,0.987330,263.984253,4.112001e+07,12859.562500,254.242264,...,33.779999,34.200001,34.610001,35.060001,35.470001,35.959999,36.520000,37.119999,37.869999,39.220001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,-11.326053,-73.359466,20190800400268919,0,6.910909,-0.226145,3012.265869,4.112006e+07,358.777405,254.962265,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3658,-11.326469,-73.359156,20190800400268920,0,6.910481,0.738749,3012.180420,4.112006e+07,1559.141113,255.196976,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3659,-11.326885,-73.358846,20190800400268921,0,6.910053,6.609153,3012.389893,4.112006e+07,-75.523026,254.653610,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3660,-11.327301,-73.358535,20190800400268922,0,6.909625,0.576247,3012.272949,4.112006e+07,807.438232,255.287933,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


C:\Users\bella\Anaconda3\envs\ndvi\lib\site-packages\pandas\core\frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,lats,long,shotNumber,quality,solar_elev,sensitivity,elevLow,delta_time,energy_total,rx_gbias,...,rh91,rh92,rh93,rh94,rh95,rh96,rh97,rh98,rh99,rh100
0,-8.487577,-75.452809,20191100400256882,1,9.819057,0.986420,256.056213,4.112001e+07,14397.813477,222.738983,...,37.570000,37.790001,38.049999,38.320000,38.619999,38.950001,39.369999,39.849998,40.450001,41.799999
1,-8.487995,-75.452505,20191100400256883,1,9.818631,0.968663,270.842468,4.112001e+07,14557.833984,223.440216,...,32.209999,32.590000,32.959999,33.410000,33.930000,34.500000,35.209999,35.990002,36.970001,38.540001
2,-8.488415,-75.452200,20191100400256884,1,9.818206,0.988050,264.018829,4.112001e+07,13634.624023,227.705154,...,37.080002,37.459999,37.900002,38.580002,39.740002,41.799999,45.919998,47.869999,49.110001,50.720001
3,-8.488835,-75.451895,20191100400256885,1,9.817778,0.988079,255.477783,4.112001e+07,16400.484375,229.080032,...,31.010000,32.250000,33.220001,34.009998,34.680000,35.400002,36.220001,37.119999,38.090000,39.930000
4,-8.489252,-75.451592,20191100400256886,1,9.817352,0.991597,269.849182,4.112001e+07,15512.489258,223.331619,...,21.650000,22.170000,23.030001,24.530001,26.740000,28.730000,30.530001,31.990000,33.369999,35.470001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3650,-11.326699,-73.365507,20191100400263624,0,6.916914,-3.674960,2967.934570,4.112006e+07,94.099899,222.635193,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3651,-11.327116,-73.365197,20191100400263625,0,6.916486,-0.743516,2967.415039,4.112006e+07,242.968887,222.572067,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3652,-11.327532,-73.364887,20191100400263626,0,6.916059,3.589182,2968.710449,4.112006e+07,-163.611603,222.440094,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3653,-11.327948,-73.364577,20191100400263627,0,6.915631,6.684797,2968.643799,4.112006e+07,-77.384171,221.987259,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


<KeysViewHDF5 ['ancillary', 'beam', 'channel', 'degrade_flag', 'delta_time', 'digital_elevation_model', 'digital_elevation_model_srtm', 'elev_highestreturn', 'elev_lowestmode', 'elevation_bias_flag', 'elevation_bin0_error', 'energy_total', 'geolocation', 'land_cover_data', 'lat_highestreturn', 'lat_lowestmode', 'latitude_bin0_error', 'lon_highestreturn', 'lon_lowestmode', 'longitude_bin0_error', 'master_frac', 'master_int', 'mean_sea_surface', 'num_detectedmodes', 'quality_flag', 'rh', 'rx_1gaussfit', 'rx_assess', 'rx_processing_a1', 'rx_processing_a2', 'rx_processing_a3', 'rx_processing_a4', 'rx_processing_a5', 'rx_processing_a6', 'selected_algorithm', 'selected_mode', 'selected_mode_flag', 'sensitivity', 'shot_number', 'solar_azimuth', 'solar_elevation', 'surface_flag']>




In [3]:
processShots()
print("HELLO")

HELLO


In [28]:
import glob
import os

csvFiles = []
for filepath in glob.glob(os.getcwd()+"/*/*/*"):
    if filepath.endswith((".csv")):
        csvFiles.append(filepath)


In [29]:

    ###READ DATA
gediL2A = h5py.File(gediFiles[5], 'r')  
beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]
print(beamNames)
print(os.getcwd())

['BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']
C:\Users\bella\OneDrive\Documents\GEDI\NaiaraPoints


In [17]:
import os
import glob
import pandas as pd

povFiles = []
for filepath in glob.glob(os.getcwd()):
    print(filepath)
    if filepath.endswith((".csv")):
        povFiles.append(filepath)
        
print(povFiles)
#combined_csv = pd.concat([pd.read_csv(f) for f in povFiles ])
#export to csv
#combined_csv.to_csv( "combinedPotapov.csv", index=False, encoding='utf-8-sig')

[]


In [23]:
import boto3
import os
s3 = boto3.resource('s3')
s3.download_file(h5file, "./")
def download_s3_folder(bucket_name, s3_folder, local_dir=None):
    """
    Download the contents of a folder directory
    Args:
        bucket_name: the name of the s3 bucket
        s3_folder: the folder path in the s3 bucket
        local_dir: a relative or absolute directory path in the local file system
    """
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=s3_folder):
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        print(obj.key)
        bucket.download_file(obj.key, target)

In [4]:
f = open("h5files.txt", "r")
realh5 = []
for line in f.readlines():
    realh5.append(line[41:])
#"s3://servir-public/gedi/peru/" 
with open('h5S3files.txt', 'w') as f:
    for item in realh5:
        print(item)
        f.write(item)

processed_GEDI02_A_2019110092939_O01996_01_T03334_02_003_01_V002.h5

processed_GEDI02_A_2019111210128_O02019_04_T04474_02_003_01_V002.h5

processed_GEDI02_A_2019114074336_O02057_01_T02477_02_003_01_V002.h5

processed_GEDI02_A_2019115191524_O02080_04_T00618_02_003_01_V002.h5

processed_GEDI02_A_2019117064711_O02103_01_T01758_02_003_01_V002.h5

processed_GEDI02_A_2019118181859_O02126_04_T01322_02_003_01_V002.h5

processed_GEDI02_A_2019121050103_O02164_01_T00748_02_003_01_V002.h5

processed_GEDI02_A_2019122163249_O02187_04_T01888_02_003_01_V002.h5

processed_GEDI02_A_2019124040434_O02210_01_T00029_02_003_01_V002.h5

processed_GEDI02_A_2019125153619_O02233_04_T05438_02_003_01_V002.h5

processed_GEDI02_A_2019128021818_O02271_01_T02018_02_003_01_V002.h5

processed_GEDI02_A_2019129135000_O02294_04_T03005_02_003_01_V002.h5

processed_GEDI02_A_2019131012144_O02317_01_T05568_02_003_01_V002.h5

processed_GEDI02_A_2019134233523_O02378_01_T03135_02_003_01_V002.h5

processed_GEDI02_A_2019136110703_O